In [1]:
import pandas as pd
from datetime import datetime, timedelta

import math

import json as json
from pandas.io.json import json_normalize

import numpy as np
import os
import scipy.optimize as sco

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
array_dataframe = []
dados = pd.DataFrame([])
scoins = []

folder_candle = 'Operacoes'

for folder in os.scandir(folder_candle):
    scoins.append(folder.name)
    for entry in os.scandir(folder):
        if entry.is_file():
            coin_symbol = (os.path.splitext(entry.name)[0])        
            with open(entry.path) as d:            
                coin_hist = json.load(d);
            df = json_normalize(coin_hist);
            
            if not df.empty:
            
                df['coin'] = folder.name
                df['amount'] = pd.to_numeric(df['amount'].replace('.',','))
                df['rate'] = pd.to_numeric(df['total'].replace('.',','))
                df['total'] = pd.to_numeric(df['total'].replace('.',','))
                df['date'] = pd.to_datetime(df['date'])            
                df['day'] = pd.to_datetime(df['date'])
                #df['day'] = pd.to_datetime(df['date'],format='%Y-%m-%d').dt.date
                
                array_dataframe.append(df)       
            else:
                print(entry.path)

dados = pd.concat(array_dataframe)

dados = dados.replace([np.inf, -np.inf], np.nan)
dados = dados.fillna(0)

dados.set_index('date', inplace=True)
dados.index = pd.to_datetime(dados.index)

dados.head()

Operacoes/USDT_BTC/2018-11.json
Operacoes/USDT_BTC/2018-12.json
Operacoes/USDT_ETH/2018-11.json
Operacoes/USDT_ETH/2018-12.json
Operacoes/USDT_LTC/2018-11.json
Operacoes/USDT_LTC/2018-12.json
Operacoes/USDT_ZEC/2018-11.json
Operacoes/USDT_ZEC/2018-12.json


,amount,globalTradeID,rate,total,tradeID,type,coin,day
date,,,,,,,,
2016-08-31 02:59:00,0.020843,51782713,12.028627,12.028627,687777,sell,USDT_BTC,2016-08-31 02:59:00
2016-08-31 02:58:46,0.020843,51782684,12.043919,12.043919,687776,buy,USDT_BTC,2016-08-31 02:58:46
2016-08-31 02:58:31,0.020843,51782678,12.028627,12.028627,687775,sell,USDT_BTC,2016-08-31 02:58:31
2016-08-31 02:58:18,0.020843,51782660,12.043919,12.043919,687774,buy,USDT_BTC,2016-08-31 02:58:18
2016-08-31 02:58:05,0.020843,51782633,12.028627,12.028627,687773,sell,USDT_BTC,2016-08-31 02:58:05


In [3]:
#Calcula a primeira data e a primeira janela de 24h

#data_inicial = dados.index.min()
data_inicial = pd.to_datetime('2015-08-01')
print(data_inicial)

#Calcula data final da primeira janela
data_final = data_inicial + timedelta(days=1)
print(data_final)

#Filtro para a janela
#mask_janela = (dados.index > data_inicial) & (dados.index <= data_final)

#Moeda base
moeda_base = 'USDT_BTC'

#Quantidade de ordens a serem avaliadas no head do book
qtd_relevante_head_book = 0.05 #5%
stop_inicial = 0.10 #10%

operacoes = []
minimo_analise = 5

id_operacoes = 0
#operacoes = dados
#operacoes['status'] = 'V'

2015-08-01 00:00:00
2015-08-02 00:00:00


In [4]:
def obtem_janela(dt_inicial, dt_final):
    return dados.loc[(dados.index > dt_inicial) & (dados.index <= dt_final)].sort_index(ascending=False)    

In [5]:
janela = obtem_janela(data_inicial,data_final)
janela.head(10)

,amount,globalTradeID,rate,total,tradeID,type,coin,day
date,,,,,,,,
2015-08-01 17:48:05,0.010000,9240823,2.798450,2.798450,19626,sell,USDT_BTC,2015-08-01 17:48:05
2015-08-01 17:43:27,0.000107,9240809,0.030301,0.030301,19625,buy,USDT_BTC,2015-08-01 17:43:27
2015-08-01 17:43:27,0.012905,9240808,3.611372,3.611372,19624,buy,USDT_BTC,2015-08-01 17:43:27
2015-08-01 17:38:59,0.224556,9240278,62.800543,62.800543,19623,buy,USDT_BTC,2015-08-01 17:38:59
2015-08-01 17:37:52,0.172513,9240246,47.566018,47.566018,19622,sell,USDT_BTC,2015-08-01 17:37:52
2015-08-01 17:37:49,0.360932,9240215,99.517826,99.517826,19621,sell,USDT_BTC,2015-08-01 17:37:49
2015-08-01 17:37:49,0.018569,9240214,5.162249,5.162249,19620,sell,USDT_BTC,2015-08-01 17:37:49
2015-08-01 17:37:49,0.000352,9240213,0.098417,0.098417,19619,sell,USDT_BTC,2015-08-01 17:37:49
2015-08-01 17:37:49,0.010130,9240212,2.834320,2.834320,19618,sell,USDT_BTC,2015-08-01 17:37:49


In [6]:
def calcula_qtd_relevante(dt):
    relevante = dt.coin.count() * qtd_relevante_head_book
    
    if relevante < minimo_analise:
        return minimo_analise
    
    return math.ceil(relevante)

def gera_id_operacao():
    global id_operacoes 
    id_operacoes = id_operacoes + 1
    return id_operacoes;

In [7]:
def avalia_moeda_base(dt):
    coin_result = dt[dt.coin == moeda_base]
    relevante = calcula_qtd_relevante(coin_result)
    
    if not coin_result.empty  :
        dt_sell = coin_result[coin_result.type == 'sell']
        dt_buy  = coin_result[coin_result.type == 'buy']
    
        #Verifica se a moeda base está em queda (sell > buy)
        if dt_sell.amount.sum() > dt_buy.amount.sum() :
            #print('Moeda base em queda')
            return True
    
    #print('Base não está em queda')
    return False


def avalia_compra(dt):
    
    #print(dt)        
    ## Verifica se a moeda base está no momento de compra
    if avalia_moeda_base(dt):
        
        for c in dt.coin.unique():
            coin_result = dt[dt.coin == c]
            
            relevante = calcula_qtd_relevante(coin_result)            
            
            coin_result = coin_result.head(relevante)
            
            if not coin_result.empty  :
                dt_sell = coin_result[coin_result.type == 'sell']
                dt_buy  = coin_result[coin_result.type == 'buy']
    
                #Verifica se a moeda base está em queda (sell > buy)
                if dt_buy.amount.sum() > dt_sell.amount.sum():
                    return True
    
    return False

In [15]:
def define_stop(dt):
    return dt.rate + (dt.rate * stop_inicial)

def simula_compra(dt):
    for c in dt[dt.coin != moeda_base].coin.unique() :        
        dt_moeda = dt[dt.coin == c]
        operacao_compra = dt[dt.coin == c].max()
        operacao_compra['id_operacao'] = gera_id_operacao()
        operacao_compra.type = 'buy'
        operacao_compra['closed'] = False
        operacao_compra['stop_update'] = 0
        operacao_compra.stop = define_stop(operacao_compra)
        operacoes.append(operacao_compra)
        #print('### COMPRA: Data/Hora:'+str(dt_moeda.day.max())+ ' Moeda: '+ str(c)+ ' Rate: '+str(dt_moeda[dt_moeda.coin == c].rate.max()))

In [29]:
def simula_venda(book_sell, operacao_buy):        
    #input("Vai Vender!")
    operacao_venda = book_sell[book_sell.coin == operacao_buy.coin].max()
    #print(book_sell)
    operacao_venda['id_operacao'] = operacao_buy.id_operacao
    operacao_buy['closed'] = True
    operacao_venda['closed'] = True
    operacoes.append(operacao_venda)
    
    #input("Vendeu!")
        
def estrategia(book):
    if not pd.DataFrame(operacoes).empty:
        
        for operacao_buy in operacoes:
            if (operacao_buy.type == 'buy') & (not operacao_buy.closed):
                coin_result = book[book.coin == operacao_buy.coin]               
                
                if not coin_result.empty  :
                    dt_sell = coin_result[coin_result.type == 'sell']
                    dt_sell = dt_sell[dt_sell.day > operacao_buy.day]
                    
                    relevante = calcula_qtd_relevante(coin_result)
                    
                    dt_sell = dt_sell.head(relevante)
    
                    #Verifica se a moeda base está em queda (sell > buy)
                    if dt_sell.rate.max() <= operacao_buy.stop:
                        simula_venda(dt_sell, operacao_buy)
                    else:
                        atualiza_stop(operacao_buy, coin_result)

def atualiza_stop(operacao, book):
    buy_book = book[book.type == 'buy']
    last_order_price = buy_book[buy_book.coin == operacao.coin].max()
    operacao.stop_update = operacao.stop_update + 1
    operacao.stop = define_stop(last_order_price)        
    #print('Preço Atualizado')
    #print(operacoes)

In [30]:
#RESET
operacoes = []
data_inicial = pd.to_datetime('2015-08-01')
data_final = data_inicial + timedelta(days=1)

#Coleta a Janela
janela = obtem_janela(data_inicial,data_final)

while not janela.empty :

    if avalia_compra(janela):
        simula_compra(janela)
    
    estrategia(janela)            
    
    data_inicial = data_final + timedelta(days=1)
    data_final = data_inicial + timedelta(days=1)
    
    #print('De: '+ str(data_inicial) + 'Até: '+ str(data_final))
    janela = obtem_janela(data_inicial,data_final)   
    #input("Press Enter to continue...")

In [31]:
dt_operacoes = pd.DataFrame(operacoes)
dt_operacoes.sort_values('id_operacao').head(40)
#dt_operacoes.head(40)
#print(operacoes)

,amount,globalTradeID,rate,total,tradeID,type,coin,day,id_operacao,closed,stop_update
0,78.055161,9506185,78.055161,78.055161,40,buy,USDT_ETH,2015-08-11 21:05:00,314,True,2.0
4,1.800000,9652099,2.700000,2.700000,112,sell,USDT_ETH,2015-08-15 17:13:39,314,True,NaN
1,64.910829,9584191,113.593950,113.593950,96,buy,USDT_ETH,2015-08-13 23:52:18,315,True,1.0
5,1.800000,9652099,2.700000,2.700000,112,sell,USDT_ETH,2015-08-15 17:13:39,315,True,NaN
2,0.044687,9577923,0.170359,0.170359,396,buy,USDT_LTC,2015-08-13 20:59:23,316,True,5.0
10,0.000254,10037842,0.000629,0.000629,439,sell,USDT_LTC,2015-08-29 21:17:11,316,True,NaN
3,13.127559,9652101,19.691338,19.691338,113,buy,USDT_ETH,2015-08-15 17:13:41,317,True,1.0
6,16.000000,9729706,21.440000,21.440000,137,sell,USDT_ETH,2015-08-17 23:25:41,317,True,NaN
7,0.123673,9996850,0.333918,0.333918,434,buy,USDT_LTC,2015-08-27 15:35:51,318,True,1.0
11,0.000254,10037842,0.000629,0.000629,439,sell,USDT_LTC,2015-08-29 21:17:11,318,True,NaN
